In [35]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from utils import prepare_dataframe
import pandas as pd
import numpy as np

In [36]:
train_df = pd.read_csv("./split-data/train_new.csv")

In [37]:
val_df = pd.read_csv("./split-data/val_new.csv")

In [38]:
test_df = pd.read_csv("./split-data/test_new.csv")

In [39]:
train_df.head()

,img_path,bbox,category,attributes
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,[ 72 79 232 273],Blouse,NaN
1,img/Sheer_Pleated-Front_Blouse/img_00000011.jpg,[ 44 10 147 270],Blouse,NaN
2,img/Sheer_Pleated-Front_Blouse/img_00000016.jpg,[ 20 86 178 276],Blouse,NaN
3,img/Sheer_Pleated-Front_Blouse/img_00000021.jpg,[ 36 76 180 255],Blouse,NaN
4,img/Sheer_Pleated-Front_Blouse/img_00000026.jpg,[ 44 74 172 237],Blouse,NaN


In [ ]:
traingen=ImageDataGenerator(rescale= 1./255 , height_shift_range=0.2, rotation_range = 10,
                           horizontal_flip = True,zoom_range= [0.7,1.3])

In [41]:
train_generator=traingen.flow_from_dataframe(
dataframe=train_df,
directory="./dataset",
x_col="img_path",
y_col= "category",
batch_size=128,
shuffle= True,
target_size = (256,256))

Found 41853 validated image filenames belonging to 46 classes.


In [42]:
valgen = ImageDataGenerator(rescale= 1./255)

In [43]:
val_generator=valgen.flow_from_dataframe(
dataframe=val_df,
directory="./dataset",
x_col="img_path",
y_col= "category",
batch_size=128,
shuffle= True,
target_size = (256,256))

Found 7972 validated image filenames belonging to 42 classes.


In [44]:
#To viualize data batches from Image Data Generator
x_batch, y_batch = next(train_generator)

#y_batch = np.expand_dims(y_batch,axis=-1)
print(x_batch.shape)
print(y_batch.shape)
print(type(x_batch))
print(type(y_batch))
print(type(x_batch[0]))
print(type(y_batch[0][0]))
print(y_batch)

'''
for i in range (0,32):
    image = x_batch[i]
    print(y_batch[i][0].shape)
    plt.imshow(image.astype(np.uint8))
    plt.show()
'''

(128, 256, 256, 3)
(128, 46)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float32'>
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


'\nfor i in range (0,32):\n    image = x_batch[i]\n    print(y_batch[i][0].shape)\n    plt.imshow(image.astype(np.uint8))\n    plt.show()\n'

PREPARING MODEL

In [45]:
from keras.models import Model
from keras.layers import Dense
from keras.regularizers import l2
from keras.optimizers import SGD,Adam
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import DirectoryIterator, ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard
from keras import backend as K

In [46]:
import tensorflow as tf

In [47]:
model_resnet = VGG16(weights='imagenet', include_top=False, pooling='avg', input_shape = (256,256,3))

In [20]:
#Setting every layer except the last 3 layers as non trainable
for layer in model_resnet.layers[:-3]:
    layer.trainable = False

In [49]:
#Branch for category classification
#We build a model in top of the resnet-50 model
x = model_resnet.output
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
y = Dense(46, activation='softmax', name='img')(x)

In [16]:
'''
#Branch for Bounding Box prediction
x_bbox = model_resnet.output
x_bbox = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x_bbox)
x_bbox = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x_bbox)
bbox = Dense(4, kernel_initializer='normal', name='bbox')(x_bbox)
'''

"\n#Branch for Bounding Box prediction\nx_bbox = model_resnet.output\nx_bbox = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x_bbox)\nx_bbox = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x_bbox)\nbbox = Dense(4, kernel_initializer='normal', name='bbox')(x_bbox)\n"

In [50]:
final_model = Model(inputs= model_resnet.input, outputs=y)

In [51]:
print(final_model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0   

In [52]:
opt = Adam(lr=0.0001, name = "Adam")

/Users/summit/miniconda3/envs/ann/lib/python3.9/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [53]:
#Compiling the final model
final_model.compile(optimizer=opt,
                    loss={'img': 'categorical_crossentropy'},
                          #'bbox': 'mean_squared_error'},
                    metrics={'img': ['accuracy', 'top_k_categorical_accuracy']}) # default: top-5
                           #  'bbox': ['mse']})

In [55]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               patience=12,
                               factor=0.5,
                               verbose=1)
tensorboard = TensorBoard(log_dir='./logs')
early_stopper = EarlyStopping(monitor='val_loss',
                              patience=30,
                              verbose=1)
checkpoint = ModelCheckpoint('./models/model.h5')

In [56]:
def custom_generator(iterator):
    while True:
        batch_x, batch_y = next(iterator)
        batch_y = np.expand_dims(y_batch,axis=-1)
        yield (batch_x,batch_y)

In [58]:
#Fitting the model on the training sata
final_model.fit(custom_generator(train_generator),steps_per_epoch=5, epochs=1, 
                validation_data = custom_generator(val_generator), validation_steps=32)

2024-12-01 14:54:46.311904: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


5/5 [==============================] - ETA: 0s - loss: 3.9698 - accuracy: 0.1984 - top_k_categorical_accuracy: 0.3906  

2024-12-01 15:01:13.980454: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


5/5 [==============================] - 1207s 286s/step - loss: 3.9698 - accuracy: 0.1984 - top_k_categorical_accuracy: 0.3906 - val_loss: 3.5451 - val_accuracy: 0.2634 - val_top_k_categorical_accuracy: 0.5854


In [59]:
testgen = ImageDataGenerator()

test_generator=testgen.flow_from_dataframe(
dataframe=test_df,
directory="./dataset",
x_col="img_path",
y_col= "category",
batch_size=128,
shuffle= True,
target_size = (256,256))

Found 8020 validated image filenames belonging to 45 classes.


In [61]:
results = final_model.evaluate(custom_generator(test_generator), steps=200)

2024-12-01 15:36:08.208549: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


KeyboardInterrupt: 

In [ ]:
# serialize model to JSON
model_json = final_model.to_json()
with open("./models/model.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
final_model.save_weights("./models/model.h5")
print("Saved model to disk")

Saved model to disk


In [31]:
print(results)

[4.113183498382568, 0.036445312201976776, 0.35445311665534973]


In [ ]:
'''
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
'''

In [33]:
print('Image loss: ' + str(results[0]))
print('Image accuracy: ' + str(results[1]))
print('Top-5 image accuracy: ' + str(results[2]))

Image loss: 4.113183498382568
Image accuracy: 0.036445312201976776
Top-5 image accuracy: 0.35445311665534973
